In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

# 커널을 구성하다보면 에러는 아니지만, 빨간색 네모 박스 warning이 뜨는 경우가 많다. 
# 그런 보기 싫은 부분들을 제거해주겠다.
# import warnings
# warnings.filterwarnings('ignore')

# os 패키지를 통해 현재 디렉토리 위치를 변경하고, read_csv를 더 편리하게 할 수 있음
import os
os.getcwd() # 현재 디렉토리 파악
# os.chdir(r"______") # 불러오고 싶은 파일이 위치한 주소를 ___에 입력

# 다른 노트북 작성할 때도 이 셀만 떼서 사용 가능하다.
import platform                

# 웬만하면 해주는 것이 좋다
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin': #맥os 사용자의 경우에
    plt.style.use('seaborn-darkgrid') 
    rc('font', family = 'AppleGothic')
    
elif platform.system() == 'Windows':#윈도우 사용자의 경우에
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    plt.style.use('seaborn-v0_8-whitegrid') # https://python-graph-gallery.com/199-matplotlib-style-sheets/
    rc('font', family=font_name)
    
# tqdm
from tqdm.notebook import tqdm

In [4]:
path = 'c:/reposit/data/dacon/2023 전력사용량 예측 AI 경진대회/'

In [3]:
df_tr = pd.read_csv(path + 'train.csv')
df_te = pd.read_csv(path + 'test.csv')
df_bd = pd.read_csv(path + 'building_info.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'c:/Windows/Fonts/malgun.ttftrain.csv'

In [ ]:
from sklearn.preprocessing import StandardScaler

# 1. 강수량(mm)의 결측치를 0으로 대체
df_tr['강수량(mm)'].fillna(0, inplace=True)

# 2. 다른 컬럼들의 결측치를 중앙값으로 대체
for column in ['풍속(m/s)', '습도(%)', '일조(hr)', '일사(MJ/m2)']:
    median_value = df_tr[column].median()
    df_tr[column].fillna(median_value, inplace=True)

# 3. 일시 컬럼에서 년, 월, 일, 시간 정보 추출
df_tr['년'] = df_tr['일시'].str.slice(0, 4).astype(int)
df_tr['월'] = df_tr['일시'].str.slice(4, 6).astype(int)
df_tr['일'] = df_tr['일시'].str.slice(6, 8).astype(int)
df_tr['시간'] = df_tr['일시'].str.slice(9, 11).astype(int)

# 4. num_date_time 컬럼 제거
df_tr.drop('num_date_time', axis=1, inplace=True)

# 5. 데이터 스케일링
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_tr[['기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)', '일사(MJ/m2)']])
scaled_df = pd.DataFrame(scaled_features, columns=['기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)', '일사(MJ/m2)'])
df_tr.update(scaled_df)

# 전처리된 데이터의 처음 5행 출력
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# 특성과 타겟 변수 분리
X = df_tr.drop(columns=['일시', '전력소비량(kWh)', '일조(hr)', '일사(MJ/m2)'])
y = df_tr['전력소비량(kWh)']

# 학습용과 검증용 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

In [ ]:
train_df = pd.merge(train_df, building_info, on='building_number', how='left')
test_df = pd.merge(test_df, building_info, on='building_number', how='left')

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

# 데이터 로드
df_tr = pd.read_csv(path + 'train.csv')
df_te = pd.read_csv(path + 'test.csv')
df_bd = pd.read_csv(path + 'building_info.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

# 조인
df_tr = pd.merge(df_tr, df_bd, on='건물번호', how='left')
df_te = pd.merge(df_te, df_bd, on='건물번호', how='left')

# 결측치 처리
df_tr['강수량(mm)'].fillna(0, inplace=True)
df_tr['풍속(m/s)'].fillna(df_tr['풍속(m/s)'].mean(), inplace=True)
df_tr['습도(%)'].fillna(df_tr['습도(%)'].mean(), inplace=True)
df_tr['일조(hr)'].fillna(0, inplace=True)
df_tr['일사(MJ/m2)'].fillna(0, inplace=True)

df_te['강수량(mm)'].fillna(0, inplace=True)
df_te['일조(hr)'] = 0
df_te['일사(MJ/m2)'] = 0

# 날짜 및 시간 특성 파생
df_tr['일시'] = pd.to_datetime(df_tr['일시'], format='%Y%m%d %H')
df_te['일시'] = pd.to_datetime(df_te['일시'], format='%Y%m%d %H')
df_tr['연'] = df_tr['일시'].dt.year
df_tr['월'] = df_tr['일시'].dt.month
df_tr['일'] = df_tr['일시'].dt.day
df_tr['시간'] = df_tr['일시'].dt.hour
df_te['연'] = df_te['일시'].dt.year
df_te['월'] = df_te['일시'].dt.month
df_te['일'] = df_te['일시'].dt.day
df_te['시간'] = df_te['일시'].dt.hour

# 필요하지 않은 컬럼 제거
df_tr.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '건물유형'], inplace=True)
df_te.drop(columns=['num_date_time', '일시', '건물유형'], inplace=True)

# 특성과 라벨 분리
X_train = df_tr.drop(columns=['전력소비량(kWh)'])
y_train = df_tr['전력소비량(kWh)']

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(df_te)

# 데이터 분할 (훈련 및 검증 세트)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [5000],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9],
}

# XGBoost 모델 생성
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# 그리드 서치 객체 생성
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_root_mean_squared_error')

# 그리드 서치 수행
grid_search.fit(X_train_split, y_train_split, eval_set=[(X_val_split, y_val_split)], early_stopping_rounds=10, verbose=True)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

# 검증 세트에서 최적 모델의 성능 평가
y_val_pred_best = grid_search.best_estimator_.predict(X_val_split)
val_rmse_best = np.sqrt(mean_squared_error(y_val_split, y_val_pred_best))
print("Validation RMSE with Best Model:", val_rmse_best)

# test 데이터에 대한 예측
y_test_pred_best = grid_search.best_estimator_.predict(X_test_scaled)
test_predictions_best = pd.DataFrame({'전력소비량(kWh)': y_test_pred_best})
test_predictions_best.to_csv('test_predictions_best.csv', index=False)

ValueError: could not convert string to float: '-'

In [5]:
# 데이터 로드
df_tr = pd.read_csv(path + 'train.csv')
df_te = pd.read_csv(path + 'test.csv')
df_bd = pd.read_csv(path + 'building_info.csv')
submission = pd.read_csv(path + 'sample_submission.csv')


In [18]:
df_bd.

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,-,-,-
96,97,호텔및리조트,55144.67,25880.00,-,-,-
97,98,호텔및리조트,53578.62,17373.75,-,-,-
98,99,호텔및리조트,53499.00,40636.00,-,-,-


In [8]:
df_tr = pd.merge(df_tr, df_bd, on='건물번호', how='left')
df_te = pd.merge(df_te, df_bd, on='건물번호', how='left')

In [19]:
df_bd.head(3)

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.0,-,-,-
1,2,건물기타,122233.47,99000.0,-,-,-
2,3,건물기타,171243.00,113950.0,40,-,-


In [25]:
df_bd['태양광용량(kW)'].astype(int)

ValueError: invalid literal for int() with base 10: '-'

In [ ]:
path = 'c:/reposit/data/dacon/2023 전력사용량 예측 AI 경진대회/'

In [26]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings(action='ignore') 

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')

#결측값을 0으로 채웁니다
train_df = train_df.fillna(0)

#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['month'] = train_df['일시'].apply(lambda x : int(x[4:6]))
train_df['day'] = train_df['일시'].apply(lambda x : int(x[6:8]))
train_df['time'] = train_df['일시'].apply(lambda x : int(x[9:11]))

train_x = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
train_y = train_df['전력소비량(kWh)']

model = RandomForestRegressor()
model.fit(train_x, train_y)

test_df['month'] = test_df['일시'].apply(lambda x : int(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : int(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : int(x[9:11]))

test_x = test_df.drop(columns=['num_date_time', '일시'])

preds = model.predict(test_x)

submission = pd.read_csv(path + 'sample_submission.csv')
submission['answer'] = preds

submission.to_csv(path + 'submission_3.csv', index=False)